<a href="https://colab.research.google.com/github/Mirra-mira/Backttracking/blob/main/Nhom05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install simpleai

In [ ]:
from simpleai.search import CspProblem, backtrack
import random
import math

def show_constraints():
    print("\n*** Ràng buộc tối thiểu ***")
    print("- Số buổi học cố định là 12 (6 ngày x 2 buổi/ngày).")
    print("- Số phòng học tối thiểu: 13 lớp cần ít nhất 2 phòng.")
    print("- Số giáo viên tối thiểu: 13 lớp cần ít nhất 2 giáo viên.")
    print("- Số lớp học không nên vượt quá (số phòng học x 12) để đảm bảo khả thi.")
    print("***************************************\n")

def schedule_classes():
    num_classes = int(input("Nhập số lượng lớp học: "))
    num_teachers = int(input("Nhập số lượng giáo viên: "))
    subjects_input = input("Nhập danh sách môn học (cách nhau bởi dấu phẩy): ")
    num_rooms = int(input("Nhập số lượng phòng học: "))

    min_rooms = math.ceil(num_classes / 12)
    min_teachers = math.ceil(num_classes / 12)
    max_classes = num_rooms * 12

    if num_rooms < min_rooms:
        print(f"Cảnh báo: Cần ít nhất {min_rooms} phòng học cho {num_classes} lớp!")
    if num_teachers < min_teachers:
        print(f"Cảnh báo: Cần ít nhất {min_teachers} giáo viên cho {num_classes} lớp!")
    if num_classes > max_classes:
        print(f"Cảnh báo: Với {num_rooms} phòng, tối đa chỉ xếp được {max_classes} lớp!")

    subjects = [s.strip() for s in subjects_input.split(',') if s.strip()]
    if not subjects:
        print("Danh sách môn học không hợp lệ!")
        return

    if num_rooms <= 0:
        print("Số lượng phòng học phải lớn hơn 0!")
        return

    rooms = [f'Phong{i}' for i in range(1, num_rooms + 1)]
    times = [
        'Sáng thứ 2', 'Chiều thứ 2',
        'Sáng thứ 3', 'Chiều thứ 3',
        'Sáng thứ 4', 'Chiều thứ 4',
        'Sáng thứ 5', 'Chiều thứ 5',
        'Sáng thứ 6', 'Chiều thứ 6',
        'Sáng thứ 7', 'Chiều thứ 7'
    ]
    classes = [f'Lop{i}' for i in range(1, num_classes + 1)]
    teachers = [f'GV{i}' for i in range(1, num_teachers + 1)]

    random.seed(42)
    class_info = {
        cls: {
            'teacher': random.choice(teachers),
            'subject': random.choice(subjects)
        }
        for cls in classes
    }

    variables = classes
    domains = {
        cls: [(time, room) for time in times for room in rooms]
        for cls in classes
    }

    constraints = []

    def constraint_different_time_room(variables, values):
        time1, room1 = values[0]
        time2, room2 = values[1]
        return (time1 != time2) or (room1 != room2)

    for i in range(len(classes)):
        for j in range(i + 1, len(classes)):
            constraints.append((constraint_different_time_room, [classes[i], classes[j]]))

    def constraint_different_teacher_time(variables, values):
        cls1, cls2 = variables
        time1, _ = values[0]
        time2, _ = values[1]
        if class_info[cls1]['teacher'] == class_info[cls2]['teacher']:
            return time1 != time2
        return True

    for i in range(len(classes)):
        for j in range(i + 1, len(classes)):
            if class_info[classes[i]]['teacher'] == class_info[classes[j]]['teacher']:
                constraints.append((constraint_different_teacher_time, [classes[i], classes[j]]))

    problem = CspProblem(variables, domains, constraints)
    solution = backtrack(problem)

    if solution:
        print("\n✅ Lịch học được sắp xếp:")
        for cls, (time, room) in solution.items():
            teacher = class_info[cls]['teacher']
            subject = class_info[cls]['subject']
            print(f"{cls} ({subject}, {teacher}): {time} tại {room}")
    else:
        print("\n❌ Không tìm được lịch học phù hợp.")

def main():
    while True:
        print("\n=== Chương trình xếp lịch học ===")
        print("1. Xem ràng buộc")
        print("2. Bắt đầu xếp lớp")
        print("0. Thoát")
        choice = input("Nhập lựa chọn (0-2): ")

        if choice == '1':
            show_constraints()
        elif choice == '2':
            schedule_classes()
        elif choice == '0':
            print("Tạm biệt!")
            break
        else:
            print("Lựa chọn không hợp lệ! Vui lòng chọn lại.")

if __name__ == "__main__":
    main()
